In [1]:
import numpy as np
import pandas as pd
import os
import pdb
from tqdm import tqdm
import scipy.stats as stats

import genomepy

import matplotlib.pyplot as plt
import seaborn as sns

import torch

In [2]:
gtex_data = pd.read_csv("GTEx_data.tsv", sep="\t")
gtex_data["CO_A"] = gtex_data.apply(lambda x: f"{x['Chr']}:{x['Exon_start']}-{x['Exon_end']}", axis=1)

gtex_data_unique_tissues = gtex_data["Tissue"].unique()
print(gtex_data_unique_tissues)

gtex_data

['Heart_Atrial_Appendage' 'Brain_Cerebellum' 'Liver']


,ID,Chr,Strand,Exon_start,Exon_end,Tissue,Change_case,Label,TrASPr_pred,Pangolin_pred,SpliceAI_pred,SpliceTF_pred,CO_A
0,vbBs-9Pl7nQ,chr3,-,33577202,33577264,Heart_Atrial_Appendage,True,0.011051,0.208662,0.006585,0.023705,0.578338,chr3:33577202-33577264
1,vbBs-9Pl7nQ,chr3,-,33577202,33577264,Brain_Cerebellum,True,0.545571,0.898411,0.007967,0.023705,0.528558,chr3:33577202-33577264
2,R0JePO6rOVA,chr5,+,14489030,14489069,Liver,False,0.156783,0.063251,0.015744,0.244254,0.128788,chr5:14489030-14489069
3,R0JePO6rOVA,chr5,+,14489030,14489069,Brain_Cerebellum,False,0.028207,0.609059,0.017516,0.244254,0.224896,chr5:14489030-14489069
4,R0JePO6rOVA,chr5,+,14489030,14489069,Heart_Atrial_Appendage,False,0.069423,0.286069,0.011704,0.244254,0.273216,chr5:14489030-14489069
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,2Kz8Ke2n-vM,chr3,+,46898378,46898472,Liver,False,0.968015,0.988131,0.609236,0.974741,0.626544,chr3:46898378-46898472
2452,FfdzmTVGYRo,chr1,+,173828333,173828399,Heart_Atrial_Appendage,False,0.918565,0.938463,0.927874,0.965602,0.925935,chr1:173828333-173828399
2453,km1JFlYp_1U,chr5,-,6631873,6631977,Brain_Cerebellum,False,0.895841,0.930458,0.873931,0.985442,0.914959,chr5:6631873-6631977
2454,Z0D1xcKqg1c,chr1,-,58663450,58663522,Heart_Atrial_Appendage,False,0.025419,0.015306,0.000708,0.144228,0.275023,chr1:58663450-58663522


In [3]:
vastdb_events = pd.read_csv("EVENT_INFO-hg38.tab.gz", sep="\t")

In [4]:
gtex_data.merge(vastdb_events, how="inner", on="CO_A")

,ID,Chr,Strand,Exon_start,Exon_end,Tissue,Change_case,Label,TrASPr_pred,Pangolin_pred,...,LE_o,FULL_CO,COMPLEX,REF_CO,LE_n,CO_C1,CO_C2,Seq_C1,Seq_A,Seq_C2
0,vbBs-9Pl7nQ,chr3,-,33577202,33577264,Heart_Atrial_Appendage,True,0.011051,0.208662,0.006585,...,63,"chr3:33581821,33577202-33577264,33576275",S,"chr3:33581821,33577202-33577264,33576275:-",63,chr3:33581821-33581928,chr3:33576169-33576275,CCCGAAGCAGTCGTATTCCTCGACCAAGTGTGAGTCAAGGATGCAG...,CCTCCAGACACCATTCCAGATCAACTGGTGCCCTCTACGCCCCCGA...,GTCCAGGTTATGGGATCAGCCAATCAAGTCGACTGTCGTCTTCTGT...
1,vbBs-9Pl7nQ,chr3,-,33577202,33577264,Brain_Cerebellum,True,0.545571,0.898411,0.007967,...,63,"chr3:33581821,33577202-33577264,33576275",S,"chr3:33581821,33577202-33577264,33576275:-",63,chr3:33581821-33581928,chr3:33576169-33576275,CCCGAAGCAGTCGTATTCCTCGACCAAGTGTGAGTCAAGGATGCAG...,CCTCCAGACACCATTCCAGATCAACTGGTGCCCTCTACGCCCCCGA...,GTCCAGGTTATGGGATCAGCCAATCAAGTCGACTGTCGTCTTCTGT...
2,BXoYu3CCW7k,chr7,-,51187914,51187958,Brain_Cerebellum,True,0.010918,0.610802,0.026915,...,45,"chr7:51190850,51187914-51187958,51184199",S,"chr7:51190850,51187914-51187958,51184199:-",45,chr7:51190850-51191078,chr7:51184102-51184199,AAATCTGTGCGTTTGGTCGTGAATTACCTGCGGACACAAAAAGCTG...,TTCGTCTGAGCAAAACTCAGTCCGAGCCTTCCCTGAGCTGTCAAG,AAACCTTTAGGAAATCATCACTTGGCAATGATGAGACAGATAAAGA...
3,BXoYu3CCW7k,chr7,-,51187914,51187958,Heart_Atrial_Appendage,True,0.378061,0.807309,0.025438,...,45,"chr7:51190850,51187914-51187958,51184199",S,"chr7:51190850,51187914-51187958,51184199:-",45,chr7:51190850-51191078,chr7:51184102-51184199,AAATCTGTGCGTTTGGTCGTGAATTACCTGCGGACACAAAAAGCTG...,TTCGTCTGAGCAAAACTCAGTCCGAGCCTTCCCTGAGCTGTCAAG,AAACCTTTAGGAAATCATCACTTGGCAATGATGAGACAGATAAAGA...
4,1E2zQLCoMeQ,chr7,-,107936279,107936373,Brain_Cerebellum,True,0.413203,0.348534,0.001366,...,95,"chr7:107937093,107936279-107936373,107935656",S,"chr7:107937093,107936279-107936373,107935656:-",95,chr7:107937093-107937277,chr7:107935415-107935656,GAAACTGATTAAAGATGTTACAGAAATGATGGCTCAAGTAGAAGTG...,AGATGGGCTCTTGCTATGCTGCTCAGGCTGGTCTTGAACTCCTGGA...,GTGCCTTGGATAGCATTACCAAGTATTTCCAGATGTCTCTTGAGGC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,AUqP23B-GyA,chr7,-,149563520,149563603,Heart_Atrial_Appendage,False,0.036779,0.139514,0.101220,...,84,"chr7:149619924,149563520-149563603,149558839",C1,"chr7:149619924,149563520-149563603,149558839:-",84,chr7:149619924-149620037,chr7:149558608-149558839,ACTACGCCATCTCCAAGTCCGAGGTCCTCTCCCAGATTGAACAAGG...,TTTAGTACTGCTAAACTAAAGTAGATTAATGAGAAGGGAGAGAGAG...,ATGCAGAAACTGAGGCCCAGCAGAGATTGAGTAACTTATGCGAGTT...
2138,2Kz8Ke2n-vM,chr3,+,46898378,46898472,Liver,False,0.968015,0.988131,0.609236,...,95,"chr3:46898192,46898378-46898472,46898662",S,"chr3:46898192,46898378-46898472,46898662:+",95,chr3:46898074-46898192,chr3:46898662-46898857,GCCATGCCTACCGACGCTGTGACCGCAATGGCAGCTGGGAGCTGGT...,GAGGTGTTTGACCGCCTGGGCATGATTTACACCGTGGGCTACTCCG...,GCGGCTGCACTGCACGCGCAACTACATCCACATGCACCTGTTCCTG...
2139,FfdzmTVGYRo,chr1,+,173828333,173828399,Heart_Atrial_Appendage,False,0.918565,0.938463,0.927874,...,67,"chr1:173826786,173828333-173828399,173830660",S,"chr1:173826786,173828333-173828399,173830660:+",67,chr1:173826687-173826786,chr1:173830660-173830761,AATTCAGTAGCTTTGTTGTCCGGACCAACACATGTGGAGAGTTGCG...,GCAAAACACATTCTTGGTCCTAAGAGATTTCGATGGGCTTGTTCAA...,TCGGCAGCCTCTGTGAAGAAGATTTTATGTGAAGCCCCTGTGGAAT...
2140,km1JFlYp_1U,chr5,-,6631873,6631977,Brain_Cerebellum,False,0.895841,0.930458,0.873931,...,105,"chr5:6632599,6631873-6631977,6625669",S,"chr5:6632599,6631873-6631977,6625669:-",105,chr5:6632599-6632756,chr5:6625564-6625669,GGCTGGGAAGGAGGCTACCCCGAGATCGTCAAGGAGAACAAGCTGT...,CCACGCAAAAGAGATTCTCCATTGCTTAAAGAACAAATATTTTAAG...,GTATCCTGAAGAACTTGCCTGGCACACAAATTTAAGTCGAAAAATC...


In [5]:
gtex_median_tissue_expression = pd.read_csv("GTEx_Analysis_v10_RNASeQCv2.4.2_gene_median_tpm.gct.gz", sep="\t", skiprows=2)

In [6]:
gtex_median_tissue_expression["gene_id"] = gtex_median_tissue_expression["Name"].str.split(".").str[0]
# gtex_median_tissue_expression = gtex_median_tissue_expression[["gene_id"] + list(gtex_data_unique_tissues)]
gtex_median_tissue_expression

,Name,Description,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Bladder,Brain_Amygdala,...,Stomach,Stomach_Mixed_Cell,Stomach_Mucosa,Stomach_Muscularis,Testis,Thyroid,Uterus,Vagina,Whole_Blood,gene_id
0,ENSG00000223972.5,DDX11L1,0.00000,0.000000,0.000000,0.0000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.167751,0.000000,0.000000,0.000000,0.00000,ENSG00000223972
1,ENSG00000227232.5,WASH7P,3.99789,3.178150,2.673080,4.0708,3.87547,3.62625,5.05094,1.45902,...,3.041170,3.328970,2.87374,4.085680,4.542470,6.317910,7.068700,5.741870,2.89503,ENSG00000227232
2,ENSG00000278267.1,MIR6859-1,0.00000,0.000000,0.000000,0.0000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,ENSG00000278267
3,ENSG00000243485.5,MIR1302-2HG,0.00000,0.000000,0.000000,0.0000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.054907,0.000000,0.000000,0.000000,0.00000,ENSG00000243485
4,ENSG00000237613.2,FAM138A,0.00000,0.000000,0.000000,0.0000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,ENSG00000237613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59028,ENSG00000198695.2,MT-ND6,3479.95000,3605.240000,6229.830000,5961.1400,5014.48000,4577.12000,8343.99000,8470.01000,...,9394.920000,11078.200000,12877.20000,17030.600000,1708.290000,2400.350000,3595.070000,2202.710000,1056.52000,ENSG00000198695
59029,ENSG00000210194.1,MT-TE,9.66208,8.599950,14.677800,18.6770,14.34600,14.35710,22.70730,21.15200,...,27.461000,32.890000,44.00160,53.812900,2.324410,4.455020,8.768250,4.251670,3.13967,ENSG00000210194
59030,ENSG00000198727.2,MT-CYB,23813.90000,25381.700000,28928.900000,13013.2000,16380.50000,13177.00000,23353.90000,32337.60000,...,27364.200000,22252.700000,31359.20000,26006.300000,15641.200000,19502.900000,15390.600000,12679.100000,2520.98000,ENSG00000198727
59031,ENSG00000210195.2,MT-TT,0.00000,0.511427,0.653953,0.0000,0.00000,0.00000,0.00000,1.33336,...,0.522499,0.424497,1.31027,0.703076,0.000000,0.000000,0.000000,0.000000,0.00000,ENSG00000210195


In [7]:
splicing_factor_expression_levels_in_knockdown_data = pd.read_parquet("splicing_factor_expression_levels.parquet")
splicing_factor_expression_levels_in_knockdown_data = splicing_factor_expression_levels_in_knockdown_data.merge(gtex_median_tissue_expression, how="left", on="gene_id")

for tissue in gtex_median_tissue_expression.columns[2:-1]:
    splicing_factor_expression_levels_in_knockdown_data[tissue] = splicing_factor_expression_levels_in_knockdown_data[tissue].fillna(0)
    splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM"] = np.log2(splicing_factor_expression_levels_in_knockdown_data[tissue] + 1)
    splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM_rel_norm"] = splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM"] / splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM"].sum()
    splicing_factor_expression_levels_in_knockdown_data = splicing_factor_expression_levels_in_knockdown_data.copy()

splicing_factor_expression_levels_in_knockdown_data

,gene_id,alias,length,AA3_log2TPM,AA3_log2RPKM,AA4_log2TPM,AA4_log2RPKM,AA5_log2TPM,AA5_log2RPKM,AA6_log2TPM,...,Testis_log2TPM,Testis_log2TPM_rel_norm,Thyroid_log2TPM,Thyroid_log2TPM_rel_norm,Uterus_log2TPM,Uterus_log2TPM_rel_norm,Vagina_log2TPM,Vagina_log2TPM_rel_norm,Whole_Blood_log2TPM,Whole_Blood_log2TPM_rel_norm
0,ENSG00000115128,SF3B6,718,8.916522,34.366343,9.430941,34.906616,9.064147,34.360251,9.209111,...,6.928347,0.004219,7.305168,0.004428,7.339244,0.004342,7.403566,0.004625,5.399718,0.004600
1,ENSG00000100813,ACIN1,4916,1.801077,26.765941,3.077200,28.373020,2.427925,27.430123,3.625946,...,6.464983,0.003937,6.877155,0.004168,6.969346,0.004123,6.599636,0.004123,5.714493,0.004868
2,ENSG00000160710,ADAR,7327,3.108120,28.383089,4.516917,29.930251,4.375503,29.603069,5.197545,...,5.993323,0.003649,6.420250,0.003891,6.826790,0.004039,6.220908,0.003886,6.665037,0.005678
3,ENSG00000197381,ADARB1,6891,1.162622,25.761565,3.497224,28.841213,2.662983,27.713861,3.719559,...,3.063242,0.001865,3.240731,0.001964,4.801748,0.002841,3.460494,0.002162,1.675843,0.001428
4,ENSG00000091542,ALKBH5,3155,4.506699,29.894607,4.069622,29.458804,3.139061,28.264033,3.790630,...,7.366313,0.004485,6.459771,0.003915,6.867699,0.004063,6.239121,0.003897,5.476712,0.004665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,ENSG00000162664,ZNF326,9717,1.020442,25.493408,2.273329,27.416758,2.113873,27.033682,2.158564,...,3.348890,0.002039,3.785132,0.002294,4.064374,0.002405,3.784567,0.002364,0.993208,0.000846
297,ENSG00000198783,ZNF830,2237,3.154406,28.435339,4.658301,30.077773,4.429997,29.660268,5.028883,...,4.802240,0.002924,4.481292,0.002716,4.635215,0.002742,4.171255,0.002606,2.791237,0.002378
298,ENSG00000212643,ZRSR1,1469,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027159,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
299,ENSG00000169249,ZRSR2,5372,0.843533,25.120819,2.351420,27.514538,0.842971,24.965541,0.925242,...,2.519552,0.001534,3.486058,0.002113,4.448828,0.002632,3.692115,0.002306,2.719258,0.002316


In [8]:
splicing_factor_expression_levels_in_knockdown_data["AV_Controls_log2TPM_rel_norm"]

0      0.006707
1      0.001846
2      0.002994
3      0.001762
4      0.002761
         ...   
296    0.001244
297    0.002872
298    0.000003
299    0.000788
300    0.000448
Name: AV_Controls_log2TPM_rel_norm, Length: 301, dtype: float64

In [9]:
splicing_factor_expression_levels_in_knockdown_data = splicing_factor_expression_levels_in_knockdown_data.copy()

for tissue in gtex_median_tissue_expression.columns[2:-1]:
    spr = stats.spearmanr(
        splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM_rel_norm"],
        splicing_factor_expression_levels_in_knockdown_data["AV_Controls_log2TPM_rel_norm"],
    )
    pr = stats.pearsonr(
        splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM_rel_norm"],
        splicing_factor_expression_levels_in_knockdown_data["AV_Controls_log2TPM_rel_norm"],
    )
    print(f"{tissue}, AV_Controls_log2TPM_rel_norm: {spr.correlation:.3f} ({spr.pvalue:.3e}), {pr[0]:.3f} ({pr[1]:.3e})")

    # for kd in splicing_factor_expression_levels_in_knockdown_data.columns:
    #     if "_log2TPM_rel_norm" in kd:            
    #         spr = stats.spearmanr(
    #             splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM_rel_norm"],
    #             splicing_factor_expression_levels_in_knockdown_data[kd],
    #         )
    #         pr = stats.pearsonr(
    #             splicing_factor_expression_levels_in_knockdown_data[tissue + "_log2TPM_rel_norm"],
    #             splicing_factor_expression_levels_in_knockdown_data[kd],
    #         )
    #         print(f"{tissue}, {kd}: {spr.correlation:.3f} ({spr.pvalue:.3e}), {pr[0]:.3f} ({pr[1]:.3e})")

Adipose_Subcutaneous, AV_Controls_log2TPM_rel_norm: 0.525 (9.314e-23), 0.567 (5.728e-27)
Adipose_Visceral_Omentum, AV_Controls_log2TPM_rel_norm: 0.547 (7.335e-25), 0.588 (2.502e-29)
Adrenal_Gland, AV_Controls_log2TPM_rel_norm: 0.555 (9.644e-26), 0.587 (2.593e-29)
Artery_Aorta, AV_Controls_log2TPM_rel_norm: 0.504 (9.093e-21), 0.556 (8.518e-26)
Artery_Coronary, AV_Controls_log2TPM_rel_norm: 0.521 (2.279e-22), 0.567 (4.995e-27)
Artery_Tibial, AV_Controls_log2TPM_rel_norm: 0.490 (1.344e-19), 0.540 (3.270e-24)
Bladder, AV_Controls_log2TPM_rel_norm: 0.490 (1.295e-19), 0.535 (1.223e-23)
Brain_Amygdala, AV_Controls_log2TPM_rel_norm: 0.532 (2.121e-23), 0.561 (2.641e-26)
Brain_Anterior_cingulate_cortex_BA24, AV_Controls_log2TPM_rel_norm: 0.514 (1.112e-21), 0.543 (1.743e-24)
Brain_Caudate_basal_ganglia, AV_Controls_log2TPM_rel_norm: 0.520 (2.997e-22), 0.549 (4.283e-25)
Brain_Cerebellar_Hemisphere, AV_Controls_log2TPM_rel_norm: 0.404 (3.037e-13), 0.435 (2.715e-15)
Brain_Cerebellum, AV_Controls_log

In [12]:
normalized_gene_expression = pd.read_parquet("normalized_gene_expression.parquet")

In [ ]:
all_tissues_median_expression_from_counts = pd.DataFrame()
all_tissues_median_expression_from_counts["gene_id"] = normalized_gene_expression["gene_id"]
all_tissues_median_expression_from_counts["length"] = normalized_gene_expression["length"]

for tissue in gtex_data_unique_tissues:
    tissue_gene_counts = pd.read_csv(f"gene_reads_v10_{tissue.lower()}.gct.gz", sep="\t", skiprows=2)
    tissue_gene_counts["gene_id"] = tissue_gene_counts["Name"].str.split(".").str[0]
    tissue_gene_counts = tissue_gene_counts.drop_duplicates(subset=["gene_id"])
    tissue_gene_counts = tissue_gene_counts.merge(normalized_gene_expression[["gene_id", "length"]], how="inner", on="gene_id")
    
    tissue_samples = []
    for col in tissue_gene_counts.columns[2:-2]:
        assert col.startswith("GTEX-")
        tissue_gene_counts[col + "_TPM"] = (tissue_gene_counts[col] / tissue_gene_counts["length"])
        rpk_sum = tissue_gene_counts[col + "_TPM"].sum()
        tissue_gene_counts[col + "_TPM"] = (tissue_gene_counts[col + "_TPM"] / rpk_sum) * 1e6
        tissue_gene_counts[col + "_log2TPM"] = np.log2(tissue_gene_counts[col + "_TPM"] + 1)
        tissue_gene_counts = tissue_gene_counts.copy()
        tissue_samples.append(col + "_log2TPM")
    tissue_gene_counts[f"{tissue}_median_expression"] = np.median(tissue_gene_counts[tissue_samples].values, axis=1)

    all_tissues_median_expression_from_counts = all_tissues_median_expression_from_counts.merge(
        tissue_gene_counts[["gene_id", f"{tissue}_median_expression"]],
        how="inner",
        on="gene_id",
    )

all_tissues_median_expression_from_counts

,gene_id,length,Heart_Atrial_Appendage_median_expression,Brain_Cerebellum_median_expression,Liver_median_expression
0,ENSG00000115128,718,6.056736,6.062578,6.316946
1,ENSG00000100813,4916,5.490581,6.849735,5.423500
2,ENSG00000160710,7327,5.116993,6.840510,5.679105
3,ENSG00000197381,6891,3.148455,6.413826,2.640859
4,ENSG00000091542,3155,6.354974,6.575708,5.700106
...,...,...,...,...,...
11884,ENSG00000198886,1377,15.937184,15.547692,16.287949
11885,ENSG00000198786,1811,13.568179,12.813789,12.993584
11886,ENSG00000198695,524,13.304314,12.656823,12.736822
11887,ENSG00000198727,1140,15.386734,14.907026,15.436086


In [19]:
all_tissues_median_expression_from_counts.drop_duplicates(subset=["gene_id"])

,gene_id,length,Heart_Atrial_Appendage_median_expression,Brain_Cerebellum_median_expression,Liver_median_expression
0,ENSG00000115128,718,6.056736,6.062578,6.316946
1,ENSG00000100813,4916,5.490581,6.849735,5.423500
2,ENSG00000160710,7327,5.116993,6.840510,5.679105
3,ENSG00000197381,6891,3.148455,6.413826,2.640859
4,ENSG00000091542,3155,6.354974,6.575708,5.700106
...,...,...,...,...,...
11884,ENSG00000198886,1377,15.937184,15.547692,16.287949
11885,ENSG00000198786,1811,13.568179,12.813789,12.993584
11886,ENSG00000198695,524,13.304314,12.656823,12.736822
11887,ENSG00000198727,1140,15.386734,14.907026,15.436086


In [18]:
normalized_gene_expression

,gene_id,alias,length,AA3_log2TPM,AA3_log2RPKM,AA4_log2TPM,AA4_log2RPKM,AA5_log2TPM,AA5_log2RPKM,AA6_log2TPM,...,ENSG00000162664_log2TPM,ENSG00000162664_log2RPKM,ENSG00000198783_log2TPM,ENSG00000198783_log2RPKM,ENSG00000212643_log2TPM,ENSG00000212643_log2RPKM,ENSG00000169249_log2TPM,ENSG00000169249_log2RPKM,AV_Controls_log2TPM,AV_Controls_log2RPKM
0,ENSG00000115128,SF3B6,718,8.916522,34.366343,9.430941,34.906616,9.064147,34.360251,9.209111,...,9.283016,34.381056,10.137775,35.845085,9.435607,34.590743,9.425161,34.543377,9.140943,34.504312
1,ENSG00000100813,ACIN1,4916,1.801077,26.765941,3.077200,28.373020,2.427925,27.430123,3.625946,...,3.727150,28.714235,3.603902,29.188673,3.407561,28.421986,3.525903,28.515198,2.516161,27.552109
2,ENSG00000160710,ADAR,7327,3.108120,28.383089,4.516917,29.930251,4.375503,29.603069,5.197545,...,4.848549,29.897943,3.619984,29.206187,5.123945,30.239188,5.337606,30.421796,4.080575,29.340132
3,ENSG00000197381,ADARB1,6891,1.162622,25.761565,3.497224,28.841213,2.662983,27.713861,3.719559,...,2.249699,27.009493,0.671566,24.954210,2.601440,27.498899,2.760684,27.650681,2.401207,27.336559
4,ENSG00000091542,ALKBH5,3155,4.506699,29.894607,4.069622,29.458804,3.139061,28.264033,3.790630,...,3.829540,28.824675,3.333525,28.891396,4.208840,29.285855,3.996804,29.023798,3.762869,29.009478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11832,ENSG00000198886,MT-ND4,1377,13.111541,38.564187,13.264398,38.742018,13.822658,39.121360,13.752427,...,13.258920,38.359131,13.421017,39.129477,13.151130,38.308191,13.203959,38.324123,13.460064,38.825876
11833,ENSG00000198786,MT-ND5,1811,10.015048,35.466462,10.648942,36.125810,10.956965,36.255041,10.718110,...,10.692825,35.792311,11.240219,36.948214,10.350507,35.506622,10.576947,35.696319,10.181663,35.546178
11834,ENSG00000198695,MT-ND6,524,8.639648,34.088836,9.476481,34.952221,9.607012,34.903963,9.073337,...,9.308639,34.406720,10.465764,36.173335,8.594617,33.748100,8.922606,34.039947,8.896802,34.259332
11835,ENSG00000198727,MT-CYB,1140,13.179536,38.632190,13.120252,38.597856,13.744088,39.042785,13.567214,...,13.135368,38.235565,13.549852,39.258323,13.176693,38.333758,13.210506,38.330671,13.331352,38.697149
